# Numerical Optimization with Python - Final Project

## Optimizing a power generation schedule

In [3]:
#Library to install
%pip install networkx seaborn pandas numpy gurobipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
#imports
import requests
import pandas as pd   
import matplotlib.pyplot as plt  
import seaborn as sns  
import warnings
import gurobipy as gp
warnings.filterwarnings("ignore")

In [ ]:
# Separation

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Create the model
model = gp.Model("generation-schedule-optimization")

# Define the variables
z = model.addVars(P, H, name="z", lb=0)  # power generated in each plant for each hour
u = model.addVars(P, H, name="u", vtype=GRB.BINARY, lb=0)  # is the plant on? for each plant and hour
v = model.addVars(P, H, name="v", vtype=GRB.BINARY)  # start up the plant? for each plant and hour
w = model.addVars(P, H, name="w", vtype=GRB.BINARY)  # shut down the plant? for each plant and hour

# Define the cost components
fuel_cost = gp.quicksum(f[i] * z[i, h] for i in P for h in H)
health_cost = gp.quicksum(a[i, h] * z[i, h] for i in P for h in H)
operating_cost = gp.quicksum(o[i] * u[i, h] for i in P for h in H)
startup_cost = gp.quicksum(s[i] * v[i, h] for i in P for h in H)
shutdown_cost = gp.quicksum(t[i] * w[i, h] for i in P for h in H)

# Set the objective function by summing all the costs
total_cost = fuel_cost + health_cost + operating_cost + startup_cost + shutdown_cost
model.setObjective(total_cost, sense=GRB.MINIMIZE)

# Define the constraints
# Demand satisfaction constraint
model.addConstrs((gp.quicksum(z[i, h] for i in P) == d[h]) for h in H)

# Minimum and maximum generation limits
model.addConstrs((z[i, h] >= m[i] * c[i] * u[i, h]) for i in P for h in H)
model.addConstrs((z[i, h] <= c[i] * u[i, h]) for i in P for h in H)

# Keep nuclear power plants always on
model.addConstrs((z[i, h] >= m[i] * c[i]) for i in P_N for h in H)

# Ramp-up and ramp-down constraints
model.addConstrs((z[i, h] - z[i, h-1] >= -r[i] * c[i]) for i in P for h in H if h > 1)
model.addConstrs((z[i, h] - z[i, h-1] <= r[i] * c[i]) for i in P for h in H if h > 1)

# Startup and shutdown logic constraints
model.addConstrs((v[i, h] <= u[i, h]) for i in P for h in H)
model.addConstrs((w[i, h] <= 1 - u[i, h]) for i in P for h in H)
model.addConstrs((v[i, h] - w[i, h] == u[i, h] - u[i, h-1]) for i in P for h in H if h > 1)

# Solve the model
model.optimize()
